In [8]:
import pandas as pd
import numpy as np
import cfbd

In [ ]:
configuration = cfbd.Configuration(
    access_token = 'E9kkZzkrUs/R8f1ZkNZUsE7SloW7yqp+JO1CStpRhLMIllwYhUa+QJjorjR1jLfo'
)

with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    games = api_instance.get_games(year=2024, season_type='both')
# Convert the response to a DataFrame for easier manipulation
    games = pd.DataFrame([game.to_dict() for game in games])

games.head()   


In [66]:
year = 2024

def get_games_data(year: int, season_type: str = 'both') -> pd.DataFrame:
    """
    Fetches games data from the CollegeFootballData API for a given year and season type.
    
    Parameters:
        year (int): The year of the season.
        season_type (str): 'regular', 'postseason', or 'both'. Defaults to 'both'.
    
    Returns:
        pd.DataFrame: A DataFrame of games data.
    """
    with cfbd.ApiClient(configuration) as api_client:
        api_instance = cfbd.GamesApi(api_client)
        games = api_instance.get_games(year=year, season_type=season_type)
        games_df = pd.DataFrame([game.to_dict() for game in games])
    return games_df

In [67]:
def get_season_player_stats(year: int, season_type: str = 'both', team: str = None) -> pd.DataFrame:
    """
    Fetches season-level player stats from the CollegeFootballData API.

    Parameters:
        year (int): The year of the season.
        season_type (str): 'regular' or 'postseason' or 'both'. Defaults to 'regular'.
        team (str, optional): Filter stats by team. Defaults to None.

    Returns:
        pd.DataFrame: A DataFrame of player season stats.
    """
    with cfbd.ApiClient(configuration) as api_client:
        api_instance = cfbd.StatsApi(api_client)
        stats = api_instance.get_player_season_stats(year=year, season_type=season_type, team=team)
        stats_df = pd.DataFrame([s.to_dict() for s in stats])
        stats_df['season'] = year  # Optional: add year tag for clarity
    return stats_df



In [ ]:
##get_season_player_stats(2024, 'both', None)

,season,playerId,player,team,conference,category,statType,stat
0,2024,-124179,Team,Lincoln (CA),Independent DII,punting,In 20,0
1,2024,-124179,Team,Lincoln (CA),Independent DII,punting,LONG,0
2,2024,-124179,Team,Lincoln (CA),Independent DII,punting,NO,1
3,2024,-124179,Team,Lincoln (CA),Independent DII,punting,TB,0
4,2024,-124179,Team,Lincoln (CA),Independent DII,punting,YDS,0
...,...,...,...,...,...,...,...,...
145763,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,SACKS,0
145764,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,SOLO,1
145765,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,TD,0
145766,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,TFL,1


In [75]:
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
all_stats = []

for year in years:
    df = get_season_player_stats(year, season_type='both')
    all_stats.append(df)

# Combine all years into one DataFrame
combined_stats = pd.concat(all_stats, ignore_index=True)


In [77]:
combined_stats

,season,playerId,player,team,conference,category,statType,stat
0,2014,-96544,Team,Lamar,Southland,rushing,CAR,1
1,2014,-96544,Team,Lamar,Southland,rushing,LONG,0
2,2014,-96544,Team,Lamar,Southland,rushing,TD,0
3,2014,-96544,Team,Lamar,Southland,rushing,YDS,-1
4,2014,-96544,Team,Lamar,Southland,rushing,YPC,-1.0
...,...,...,...,...,...,...,...,...
1142183,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,SACKS,0
1142184,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,SOLO,1
1142185,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,TD,0
1142186,2024,5257802,Fred Davis Ii,Jacksonville State,Conference USA,defensive,TFL,1


In [81]:
combined_stats['stat_key'] = combined_stats['category'] + '_' + combined_stats['statType']

wide_stats = combined_stats.pivot_table(
    index=['season', 'playerId', 'player', 'team', 'conference'],
    columns='stat_key',
    values='stat',
    aggfunc='first'  # or 'sum' if you expect multiple entries per stat
).reset_index()

wide_stats.columns.name = None  # remove MultiIndex name
wide_stats.fillna(0, inplace=True)

cfb_player_stats  = wide_stats
cfb_player_stats


,season,playerId,player,team,conference,defensive_PD,defensive_QB HUR,defensive_SACKS,defensive_SOLO,defensive_TD,...,receiving_LONG,receiving_REC,receiving_TD,receiving_YDS,receiving_YPR,rushing_CAR,rushing_LONG,rushing_TD,rushing_YDS,rushing_YPC
0,2014,-12171,Team,Southern Illinois,MVFC,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2014,-15611,Team,SE Louisiana,Southland,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2014,-5,Team,UAB,Conference USA,0,0,0,0,0,...,0,0,0,0,0,6,0,0,-9,-1.5
3,2014,-5136,Team,Abilene Christian,Southland,0,0,0,0,0,...,0,0,0,0,0,3,0,0,-5,-1.7
4,2014,-5151,Team,Akron,Mid-American,0,0,0,0,0,...,0,0,0,0,0,3,0,0,-4,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120430,2024,553074,Aaron Young,Old Dominion,Sun Belt,0,0,0,0,0,...,8,7,0,32,4.6,149,51,6,768,5.2
120431,2024,560145,Mike Newman,Western Illinois,Big South-OVC,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
120432,2024,560147,Kevin Johnson,Hampton,CAA,0,0,0,0,0,...,31,12,1,131,10.9,0,0,0,0,0
120433,2024,560240,Eric Johnson,Idaho State,Big Sky,0,0,0,0,0,...,19,11,2,108,9.8,0,0,0,0,0


In [82]:
from sqlalchemy import create_engine

username = "postgres"
password = "postgres"
host = "localhost"
port = "5432"  # <-- Use the correct port here
database = "nfl_stats"

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

In [83]:
cfb_player_stats.to_sql("cfb_player_stats", con=engine, if_exists="replace", index=False)

217